# T1D Data Process with Low Complexity Filter
Starting with previously trimmed fastq files.

In [4]:
import pysam
import pymongo
from glob import glob
import gzip
import re
import Bio
from Bio import SeqIO
import os
import pandas as pd
from joblib import Parallel, delayed
from multiprocessing import cpu_count
import numpy as np

In [14]:
%%bash
for f in *.trim.fastq.gz; do
out="dust.fastq.gz"
echo "fqtrim -l 15 -D -p 12 -o $out $f"
fqtrim -l 15 -D -p 12 -o $out $f
done

fqtrim -l 15 -D -p 12 -o dust.fastq.gz S01.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S02.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S03.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S04.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S05.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S06.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S07.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S08.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S09.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S10.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S11.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S12.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S13.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S14.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S15.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S16.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.fastq.gz S17.trim.fastq.gz
fqtrim -l 15 -D -p 12 -o dust.f

In [15]:
%%bash
for f in *.trim.dust.fastq.gz; do
out=${f/.trim.dust.fastq.gz/.bam}
echo "bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U $f | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o $out";
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U $f | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o $out ;
done

bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S01.trim.dust.fastq.gz | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o S01.bam
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S02.trim.dust.fastq.gz | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o S02.bam
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S03.trim.dust.fastq.gz | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o S03.bam
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S04.trim.dust.fastq.gz | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o S04.bam
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S05.trim.dust.fastq.gz | samtools view - -Sb  -h -t /avicenna/genomes/hg38/hg38.fa.fai -o S05.bam
bowtie2 --sensitive --end-to-end -N 1 -p 12 -x /avicenna/genomes/hg38/hg38 -U S06.trim.dust.fastq.gz | samtools view - -

1015469 reads; of these:
  1015469 (100.00%) were unpaired; of these:
    229985 (22.65%) aligned 0 times
    236877 (23.33%) aligned exactly 1 time
    548607 (54.02%) aligned >1 times
77.35% overall alignment rate
3269311 reads; of these:
  3269311 (100.00%) were unpaired; of these:
    740256 (22.64%) aligned 0 times
    630427 (19.28%) aligned exactly 1 time
    1898628 (58.07%) aligned >1 times
77.36% overall alignment rate
2422437 reads; of these:
  2422437 (100.00%) were unpaired; of these:
    1121984 (46.32%) aligned 0 times
    225647 (9.31%) aligned exactly 1 time
    1074806 (44.37%) aligned >1 times
53.68% overall alignment rate
2667977 reads; of these:
  2667977 (100.00%) were unpaired; of these:
    1543527 (57.85%) aligned 0 times
    139284 (5.22%) aligned exactly 1 time
    985166 (36.93%) aligned >1 times
42.15% overall alignment rate
1872931 reads; of these:
  1872931 (100.00%) were unpaired; of these:
    744989 (39.78%) aligned 0 times
    151867 (8.11%) aligned e

In [16]:
%%bash
for f in *.bam; do
out=${f/.bam/.srt.bam};
echo "samtools sort -o $out $f";
samtools sort -@ 4 -m 2G -o $out $f
samtools index $out;
done

samtools sort -o S01.srt.bam S01.bam
samtools sort -o S02.srt.bam S02.bam
samtools sort -o S03.srt.bam S03.bam
samtools sort -o S04.srt.bam S04.bam
samtools sort -o S05.srt.bam S05.bam
samtools sort -o S06.srt.bam S06.bam
samtools sort -o S07.srt.bam S07.bam
samtools sort -o S08.srt.bam S08.bam
samtools sort -o S09.srt.bam S09.bam
samtools sort -o S10.srt.bam S10.bam
samtools sort -o S11.srt.bam S11.bam
samtools sort -o S12.srt.bam S12.bam
samtools sort -o S13.srt.bam S13.bam
samtools sort -o S14.srt.bam S14.bam
samtools sort -o S15.srt.bam S15.bam
samtools sort -o S16.srt.bam S16.bam
samtools sort -o S17.srt.bam S17.bam
samtools sort -o S18.srt.bam S18.bam
samtools sort -o S19.srt.bam S19.bam
samtools sort -o S20.srt.bam S20.bam


[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory bl

In [18]:
%%bash
for f in *.srt.bam; do
out=${f/.srt.bam/.srt.dd.bam};
echo "umi_tools dedup -I $f -S $out";
umi_tools dedup -I $f -S $out &
done

umi_tools dedup -I S01.srt.bam -S S01.srt.dd.bam
umi_tools dedup -I S02.srt.bam -S S02.srt.dd.bam
umi_tools dedup -I S03.srt.bam -S S03.srt.dd.bam
umi_tools dedup -I S04.srt.bam -S S04.srt.dd.bam
umi_tools dedup -I S05.srt.bam -S S05.srt.dd.bam
umi_tools dedup -I S06.srt.bam -S S06.srt.dd.bam
umi_tools dedup -I S07.srt.bam -S S07.srt.dd.bam
umi_tools dedup -I S08.srt.bam -S S08.srt.dd.bam
umi_tools dedup -I S09.srt.bam -S S09.srt.dd.bam
umi_tools dedup -I S10.srt.bam -S S10.srt.dd.bam
umi_tools dedup -I S11.srt.bam -S S11.srt.dd.bam
umi_tools dedup -I S12.srt.bam -S S12.srt.dd.bam
umi_tools dedup -I S13.srt.bam -S S13.srt.dd.bam
umi_tools dedup -I S14.srt.bam -S S14.srt.dd.bam
umi_tools dedup -I S15.srt.bam -S S15.srt.dd.bam
umi_tools dedup -I S16.srt.bam -S S16.srt.dd.bam
umi_tools dedup -I S17.srt.bam -S S17.srt.dd.bam
umi_tools dedup -I S18.srt.bam -S S18.srt.dd.bam
umi_tools dedup -I S19.srt.bam -S S19.srt.dd.bam
umi_tools dedup -I S20.srt.bam -S S20.srt.dd.bam
# UMI-tools version:

In [23]:
%%bash
for f in *.srt.dd.bam; do
out=${f/.srt.dd.bam/.sort.dd.bam};
echo "samtools sort -o $out $f";
samtools sort -@ 4 -m 2G -o $out $f
samtools index $out;
done

samtools sort -o S01.sort.dd.bam S01.srt.dd.bam
samtools sort -o S02.sort.dd.bam S02.srt.dd.bam
samtools sort -o S03.sort.dd.bam S03.srt.dd.bam
samtools sort -o S04.sort.dd.bam S04.srt.dd.bam
samtools sort -o S05.sort.dd.bam S05.srt.dd.bam
samtools sort -o S06.sort.dd.bam S06.srt.dd.bam
samtools sort -o S07.sort.dd.bam S07.srt.dd.bam
samtools sort -o S08.sort.dd.bam S08.srt.dd.bam
samtools sort -o S09.sort.dd.bam S09.srt.dd.bam
samtools sort -o S10.sort.dd.bam S10.srt.dd.bam
samtools sort -o S11.sort.dd.bam S11.srt.dd.bam
samtools sort -o S12.sort.dd.bam S12.srt.dd.bam
samtools sort -o S13.sort.dd.bam S13.srt.dd.bam
samtools sort -o S14.sort.dd.bam S14.srt.dd.bam
samtools sort -o S15.sort.dd.bam S15.srt.dd.bam
samtools sort -o S16.sort.dd.bam S16.srt.dd.bam
samtools sort -o S17.sort.dd.bam S17.srt.dd.bam
samtools sort -o S18.sort.dd.bam S18.srt.dd.bam
samtools sort -o S19.sort.dd.bam S19.srt.dd.bam
samtools sort -o S20.sort.dd.bam S20.srt.dd.bam


[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[bam_sort_core] merging from 0 files and 4 in-memory bl

In [24]:
def processInput(path):
    samfile = pysam.AlignmentFile(path, "rb")
    data = samfile.fetch()
    sample_name = re.split("\.",re.split("/", path)[-1])[0]
    documents = {}
    for read in data:
        mapflag = False
        mapq = read.mapping_quality
        if mapq<15:
            mapflag = True
        seq = read.get_forward_sequence()
        ref_id = read.reference_name
        ref_start = read.reference_start
        ref_end = read.reference_end
        strand = "+"
        if read.is_reverse:
            strand = "-"
        if seq in documents and documents[seq]["strand"] == strand:
            seq_dict = documents[seq]
            seq_dict["count"] += 1
            if mapflag and not seq_dict["mapflag"]:
                seq_dict["mapflag"] = mapflag
        else:
            documents[seq] = {"seq": seq, "count":1, 
                             "ref_id": ref_id, "start": ref_start,
                              "end": ref_end, "strand": strand,
                              "len" : len(seq),
                              "mapflag": mapflag}
    df = pd.DataFrame.from_dict(documents,orient="index")
    df = df.sort_values(by="count", ascending=False)
    df.index = pd.RangeIndex(len(df.index))
    df.to_csv(f"{sample_name}_data.csv")
    return df

In [27]:
path="./"
bamfiles = []
for subf in os.scandir(path):
    if subf.name.endswith(".sort.dd.bam"):
        bamfiles.append(subf.path)

In [28]:
num_cores = cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in bamfiles)